<a href="https://colab.research.google.com/github/nisanuro/CNG562-Assignment-3/blob/master/CNG562_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score
from sklearn import metrics, datasets, preprocessing
from sklearn.datasets import load_breast_cancer 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN

%matplotlib inline

In [0]:
def correlation_map(df):
    plt.figure(figsize=(20,12)) 

    corr = df.corr()
    sns.heatmap(corr, annot=True, cmap='hot')
    plt.show()

In [0]:
def filter_features(data, feature_indexes):
	# eliminate above column indices from the data and return new set
	filtered_data = np.delete(data, feature_indexes, axis=1)

	return filtered_data

In [0]:
def fourError(X, Y, model, r, future_scaling):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state = r, stratify=Y)
    
    if(future_scaling):
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
    
    Train_x, TrainDev_x, Train_y, TrainDev_y = train_test_split(X_train, Y_train, test_size=0.2, random_state=0, stratify=Y_train)
    Dev_x, Test_x, Dev_y, Test_y = train_test_split(X_test, Y_test, test_size=0.5, random_state=0, stratify=Y_test)

    model.fit(Train_x, Train_y)

    y_true, trainDev_pred = TrainDev_y, model.predict(TrainDev_x)

    print("\nTrain-Train Dev,   e1:", metrics.mean_squared_error(TrainDev_y, trainDev_pred))
    print("Accuracy: ", 1 - metrics.mean_squared_error(TrainDev_y, trainDev_pred),"\n")

    y_true, dev_pred = Dev_y, model.predict(Dev_x)
    print("Train-Dev,   e2", metrics.mean_squared_error(Dev_y, dev_pred))
    print("Accuracy: ", 1 - metrics.mean_squared_error(Dev_y, dev_pred),"\n")

    y_true, test_pred = Test_y, model.predict(Test_x)
    print("Train-Test,   e3: ", metrics.mean_squared_error(Test_y, test_pred))
    print("Accuracy: ", 1 - metrics.mean_squared_error(Test_y, test_pred),"\n")

    y_true, devTest_pred = Y_test, model.predict(X_test)
    print("Train-(Dev+Test),   e4: ", metrics.mean_squared_error(Y_test, devTest_pred))
    print("Accuracy: ", 1 - metrics.mean_squared_error(Y_test, devTest_pred),"\n")